In [1]:
import torch
import datetime
from torch.autograd import Variable
from torchvision.models import resnet, alexnet, inception_v3
from torchvision.datasets import coco
from torchvision import transforms

In [2]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

In [3]:
BASE = '/Users/ankitmathur/'
ANN_PATH = 'datasets/coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = 'datasets/coco/cocoapi/images/val2017/'

In [4]:
def merge_coco(l):
    return torch.stack([x[0] for x in l])

## ResNet

In [5]:
resnet_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
resnet_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=resnet_transform)
resnet_eval_loader = torch.utils.data.DataLoader(resnet_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.67s)
creating index...
index created!


In [6]:
resnet_model = resnet.resnet18(pretrained=True)

In [7]:
def resnet_measure_forward(model, x):
    print('Image size:', compute_mem_size(x))
    t0 = datetime.datetime.now()
    x = model.conv1(x)
    x = model.bn1(x)
    x = model.relu(x)
    t1 = datetime.datetime.now()
    dt1 = t1 - t0
    print('After conv1 (' + str(dt1.total_seconds()) + '):', compute_mem_size(x))
    t2 = datetime.datetime.now()
    x = model.maxpool(x)
    t3 = datetime.datetime.now()
    dt2 = t3 - t2
    print('After maxpool (' + str(dt2.total_seconds()) + '):', compute_mem_size(x))
    t4 = datetime.datetime.now()
    x = model.layer1(x)
    t5 = datetime.datetime.now()
    dt3 = t5 - t4
    print('After l1 (' + str(dt3.total_seconds()) + '):', compute_mem_size(x))
    t6 = datetime.datetime.now()
    x = model.layer2(x)
    t7 = datetime.datetime.now()
    dt4 = t7 - t6
    print('After l2 (' + str(dt4.total_seconds()) + '):', compute_mem_size(x))
    t8 = datetime.datetime.now()
    x = model.layer3(x)
    t9 = datetime.datetime.now()
    dt5 = t9 - t8
    print('After l3 (' + str(dt5.total_seconds()) + '):', compute_mem_size(x))
    t10 = datetime.datetime.now()
    x = model.layer4(x)
    t11 = datetime.datetime.now()
    dt6 = t11 - t10
    print('After l4 (' + str(dt6.total_seconds()) + '):', compute_mem_size(x))
    t12 = datetime.datetime.now()
    x = model.avgpool(x)
    t13 = datetime.datetime.now()
    dt7 = t13 - t12
    print('After avgpool (' + str(dt7.total_seconds()) + '):', compute_mem_size(x))
    x = x.view(x.size(0), -1)
    t14 = datetime.datetime.now()
    x = model.fc(x)
    t15 = datetime.datetime.now()
    dt8 = t15 - t14
    print('After fc (' + str(dt8.total_seconds()) + '):', compute_mem_size(x))
    return x

In [8]:
resnet_model.eval()
for batch in resnet_eval_loader:
    resnet_measure_forward(resnet_model, batch)
    break

Image size: 602112.0
After conv1 (0.010392): 3211264.0
After maxpool (0.007877): 802816.0
After l1 (0.024636): 802816.0
After l2 (0.020516): 401408.0
After l3 (0.016048): 200704.0
After l4 (0.018864): 100352.0
After avgpool (0.000272): 2048.0
After fc (0.00098): 4000.0


## AlexNet

In [9]:
alexnet_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
alexnet_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=resnet_transform)
alexnet_eval_loader = torch.utils.data.DataLoader(alexnet_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.90s)
creating index...
index created!


In [10]:
alexnet_model = alexnet(pretrained=True)

In [11]:
def alexnet_measure_forward(model, x):
    print('Image size: ', compute_mem_size(x))
    for module in model.features:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
    # resize step.
    x = x.view(x.size(0), 256 * 6 * 6)
    # classification step
    for module in model.classifier:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))

In [12]:
alexnet_model.eval()
for batch in alexnet_eval_loader:
    alexnet_measure_forward(alexnet_model, batch)
    break

Image size:  602112.0
After Conv2d(0.005755):  774400.0
After ReLU(0.000597):  774400.0
After MaxPool2d(0.001909):  186624.0
After Conv2d(0.005155):  559872.0
After ReLU(0.000368):  559872.0
After MaxPool2d(0.001489):  129792.0
After Conv2d(0.002306):  259584.0
After ReLU(0.000209):  259584.0
After Conv2d(0.003284):  173056.0
After ReLU(0.000167):  173056.0
After Conv2d(0.001712):  173056.0
After ReLU(0.00018):  173056.0
After MaxPool2d(0.000455):  36864.0
After Dropout(8.8e-05):  36864.0
After Linear(0.009772):  16384.0
After ReLU(0.00029):  16384.0
After Dropout(5.5e-05):  16384.0
After Linear(0.004322):  16384.0
After ReLU(0.000324):  16384.0
After Linear(0.00124):  4000.0


## Inception V3

In [13]:
inception_transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225])
])
inception_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=resnet_transform)
inception_eval_loader = torch.utils.data.DataLoader(alexnet_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.86s)
creating index...
index created!


In [14]:
inception_model = inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /Users/ankitmathur/.torch/models/inception_v3_google-1a9a5a14.pth
100.0%


In [ ]:
# incomplete
def inception_measure_forward(model, x):
    # 299 x 299 x 3
    x = self.Conv2d_1a_3x3(x)
    # 149 x 149 x 32
    x = self.Conv2d_2a_3x3(x)
    # 147 x 147 x 32
    x = self.Conv2d_2b_3x3(x)
    # 147 x 147 x 64
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    # 73 x 73 x 64
    x = self.Conv2d_3b_1x1(x)
    # 73 x 73 x 80
    x = self.Conv2d_4a_3x3(x)
    # 71 x 71 x 192
    x = F.max_pool2d(x, kernel_size=3, stride=2)
    # 35 x 35 x 192
    x = self.Mixed_5b(x)
    # 35 x 35 x 256
    x = self.Mixed_5c(x)
    # 35 x 35 x 288
    x = self.Mixed_5d(x)
    # 35 x 35 x 288
    x = self.Mixed_6a(x)
    # 17 x 17 x 768
    x = self.Mixed_6b(x)
    # 17 x 17 x 768
    x = self.Mixed_6c(x)
    # 17 x 17 x 768
    x = self.Mixed_6d(x)
    # 17 x 17 x 768
    x = self.Mixed_6e(x)
    # 17 x 17 x 768
    x = self.Mixed_7a(x)
    # 8 x 8 x 1280
    x = self.Mixed_7b(x)
    # 8 x 8 x 2048
    x = self.Mixed_7c(x)
    # 8 x 8 x 2048
    x = F.avg_pool2d(x, kernel_size=8)
    # 1 x 1 x 2048
    x = F.dropout(x, training=self.training)
    # 1 x 1 x 2048
    x = x.view(x.size(0), -1)
    # 2048
    x = self.fc(x)
    # 1000 (num_classes)
    return x